In [3]:
import numpy as np
import pandas as pd
import csv
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, classification_report, accuracy_score
import pickle
import xgboost as xgb
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV

In [4]:
loaded_df=pd.read_csv('flights.csv')
loaded_df.head()

/tmp/ipykernel_2681743/1050476599.py:1: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  loaded_df=pd.read_csv('flights.csv')


,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,...,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,...,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,...,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,...,756.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,...,259.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df = loaded_df.copy()

In [7]:
df = df[df['CANCELLED'] == 0]
# Columns to drop
columns_to_drop = [
    "YEAR", "TAIL_NUMBER", "CANCELLED", "CANCELLATION_REASON",
    "AIR_SYSTEM_DELAY", "SECURITY_DELAY", "AIRLINE_DELAY", "LATE_AIRCRAFT_DELAY", "WEATHER_DELAY"
]

# Drop columns
df.drop(columns=columns_to_drop, inplace=True)

columns_to_drop = ['SCHEDULED_DEPARTURE','WHEELS_ON','TAXI_IN','DIVERTED']
df.drop(columns=columns_to_drop, inplace=True)

columns_to_drop = ['SCHEDULED_TIME','ELAPSED_TIME','ARRIVAL_TIME']
df.drop(columns=columns_to_drop, inplace=True)

In [8]:
df.dropna(inplace=True)
df.shape

(5714008, 15)

In [9]:
import datetime

# List of holidays in 2015 (MONTH, DAY)
holidays_2015 = [
    datetime.date(2015, 1, 1),   # New Year's Day
    datetime.date(2015, 1, 19),  # Martin Luther King Jr. Day
    datetime.date(2015, 2, 16),  # Presidents' Day
    datetime.date(2015, 5, 25),  # Memorial Day
    datetime.date(2015, 7, 3),   # Independence Day (observed)
    datetime.date(2015, 9, 7),   # Labor Day
    datetime.date(2015, 10, 12), # Columbus Day
    datetime.date(2015, 11, 11), # Veterans Day
    datetime.date(2015, 11, 26), # Thanksgiving Day
    datetime.date(2015, 12, 25)  # Christmas Day
]

# Create a new column with the date
df["DATE"] = df.apply(lambda row: datetime.date(2015, row["MONTH"], row["DAY"]), axis=1)


# Include one day before and after each holiday
expanded_holidays = set(holidays_2015)
for holiday in holidays_2015:
    expanded_holidays.add(holiday - datetime.timedelta(days=1))  # Day before
    expanded_holidays.add(holiday + datetime.timedelta(days=1))  # Day after

# Update is_holiday with expanded holidays
df["is_holiday"] = df["DATE"].apply(lambda x: 1 if x in expanded_holidays else 0)
df.drop("DATE", axis=1, inplace=True)
print(df["is_holiday"])

0          1
1          1
2          1
3          1
4          1
          ..
5819074    0
5819075    0
5819076    0
5819077    0
5819078    0
Name: is_holiday, Length: 5714008, dtype: int64


In [10]:
df.info
df["DEPARTURE_TIME_HOUR"] = df["DEPARTURE_TIME"] // 100
df["DEPARTURE_TIME_MINUTE"] = df["DEPARTURE_TIME"] % 100
df.drop("DEPARTURE_TIME", axis=1, inplace=True)

df["SCHEDULED_ARRIVAL_HOUR"] = df["SCHEDULED_ARRIVAL"] // 100
df["SCHEDULED_ARRIVAL_MINUTE"] = df["SCHEDULED_ARRIVAL"] % 100
df.drop("SCHEDULED_ARRIVAL", axis=1, inplace=True)

In [12]:
df.to_csv("everything_done_except_onehot.csv")

In [13]:
df_sampled2 = pd.get_dummies(df, columns=["AIRLINE"], prefix="AIRLINE", drop_first=True)

# Only convert the one-hot encoded columns to int
airline_columns = [col for col in df_sampled2.columns if col.startswith("AIRLINE_")]
df_sampled2[airline_columns] = df_sampled2[airline_columns].astype(int)

In [16]:
# Compute frequency maps
airport_freq_map = df_sampled2["ORIGIN_AIRPORT"].value_counts().to_dict()
print(airport_freq_map)

{'ATL': 343506, 'ORD': 276554, 'DFW': 232647, 'DEN': 193402, 'LAX': 192003, 'PHX': 145552, 'SFO': 145491, 'IAH': 144019, 'LAS': 131937, 'MSP': 111055, 'SEA': 110178, 'MCO': 109532, 'DTW': 106992, 'BOS': 104804, 'CLT': 99052, 'EWR': 98341, 'SLC': 96505, 'LGA': 94834, 'JFK': 91663, 'BWI': 84329, 'MDW': 78685, 'FLL': 72580, 'DCA': 71579, 'SAN': 69270, 'MIA': 68301, 'PHL': 64751, 'TPA': 63077, 'DAL': 58672, 'HOU': 51132, 'PDX': 46777, 'BNA': 46371, 'STL': 46181, 'HNL': 42946, 'OAK': 41669, 'AUS': 41392, 'MSY': 38292, 'MCI': 38106, 'SJC': 37708, 'SMF': 36788, 'SNA': 36708, 'CLE': 33765, 'IAD': 33664, 'RDU': 31074, 10397: 30750, 'SAT': 29375, 'MKE': 29172, 'RSW': 26938, 13930: 26046, 'IND': 25048, 'SJU': 24399, 'PIT': 23669, 'CMH': 23557, 'PBI': 22134, 'OGG': 20477, 'CVG': 19467, 11298: 19449, 'ABQ': 18918, 'BUR': 18527, 'BDL': 18375, 'JAX': 17777, 'ONT': 17769, 11292: 17083, 12892: 16626, 'BUF': 16476, 'OMA': 16442, 'ANC': 15818, 'OKC': 15797, 'TUS': 14922, 'RIC': 14887, 'MEM': 14381, 'TUL'

In [17]:
from sklearn.model_selection import train_test_split

X = df_sampled2.drop("ARRIVAL_DELAY", axis=1)  # Features
y = df_sampled2["ARRIVAL_DELAY"]

# Split into train+validation and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X, y, test_size=0.1, random_state=42
)

# Split train+validation into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.11, random_state=42  # 0.1765 = 10 / 90
)

print(f"Train set: {X_train.shape}, Validation set: {X_val.shape}, Test set: {X_test.shape}")

Train set: (4576920, 29), Validation set: (565687, 29), Test set: (571401, 29)


In [18]:
print(f"Train set: {X_train.shape}, Validation set: {X_val.shape}, Test set: {X_test.shape}")

Train set: (4576920, 29), Validation set: (565687, 29), Test set: (571401, 29)


In [22]:
# Frequency encoding for train, validation, and test sets
X_train["ORIGIN_AIRPORT_freq"] = X_train["ORIGIN_AIRPORT"].map(airport_freq_map)
X_val["ORIGIN_AIRPORT_freq"] = X_val["ORIGIN_AIRPORT"].map(airport_freq_map)
X_test["ORIGIN_AIRPORT_freq"] = X_test["ORIGIN_AIRPORT"].map(airport_freq_map)

X_train["DESTINATION_AIRPORT_freq"] = X_train["DESTINATION_AIRPORT"].map(airport_freq_map)
X_val["DESTINATION_AIRPORT_freq"] = X_val["DESTINATION_AIRPORT"].map(airport_freq_map)
X_test["DESTINATION_AIRPORT_freq"] = X_test["DESTINATION_AIRPORT"].map(airport_freq_map)

In [23]:
# Drop the original airport code columns
X_train = X_train.drop(["ORIGIN_AIRPORT", "DESTINATION_AIRPORT"], axis=1)
X_val = X_val.drop(["ORIGIN_AIRPORT", "DESTINATION_AIRPORT"], axis=1)
X_test = X_test.drop(["ORIGIN_AIRPORT", "DESTINATION_AIRPORT"], axis=1)

In [25]:

def run_xgboost(X_train, y_train, X_val, y_val, params):

  # Initialize XGBoost Regressor
  xgb_model = xgb.XGBRegressor(
      tree_method="gpu_hist",
      objective=params['objective'],  # For regression
      n_estimators=params['n_estimators'],             # Number of trees
      max_depth=params['max_depth'],                  # Maximum depth of each tree
      learning_rate=params['learning_rate'],            # Learning rate (eta)
      subsample=params['subsample'],                # Fraction of samples used per tree
      colsample_bytree=params['colsample_bytree'],         # Fraction of features used per tree
      random_state=42
  )

  # Train the model
  xgb_model.fit(X_train, y_train)

  # Predict on the validation set
  y_val_pred = xgb_model.predict(X_val)

  # Evaluate the model
  mae = mean_absolute_error(y_val, y_val_pred)
  mse = mean_squared_error(y_val, y_val_pred)
  r2 = r2_score(y_val, y_val_pred)

  print("Validation Set Performance:")
  print(f"Mean Absolute Error (MAE): {mae:.2f}")
  print(f"Mean Squared Error (MSE): {mse:.2f}")
  print(f"R-squared (R²): {r2:.2f}")


  # Save the XGBoost model
  xgb_model.save_model("xgb_model.json")
  print("Model saved as xgb_model.json")

    # Predict using the loaded model
  y_test_pred = xgb_model.predict(X_test)

  mae = mean_absolute_error(y_test, y_test_pred)
  mse = mean_squared_error(y_test, y_test_pred)
  r2 = r2_score(y_test, y_test_pred)

  print("\n\n\nFinal Test Set Performance:")
  print(f"Mean Absolute Error (MAE): {mae:.2f}")
  print(f"Mean Squared Error (MSE): {mse:.2f}")
  print(f"R-squared (R²): {r2:.2f}")


In [26]:
params = {
    "objective" :"reg:squarederror",  # For regression
    "n_estimators" : 500,             # Number of trees
    "max_depth" : 15,                  # Maximum depth of each tree
    "learning_rate" : 0.01,            # Learning rate (eta)
    "subsample" : 0.8,                # Fraction of samples used per tree
    "colsample_bytree" : 0.8,         # Fraction of features used per tree
}
run_xgboost(X_train, y_train, X_val, y_val,params)

/home/csgrads/shahr072/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [16:41:19] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/home/csgrads/shahr072/.local/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [16:43:13] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


Validation Set Performance:
Mean Absolute Error (MAE): 5.55
Mean Squared Error (MSE): 76.76
R-squared (R²): 0.95
Model saved as xgb_model.json



Final Test Set Performance:
Mean Absolute Error (MAE): 5.54
Mean Squared Error (MSE): 74.34
R-squared (R²): 0.95
